<a href="https://colab.research.google.com/github/sergiolucero/notebooks/blob/main/Spotipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Spotify to La Cuerda
import os
os.environ['SPOTIPY_CLIENT_ID'] = 'd843f4fe52d7433faf04e4e15b27ae3a'
os.environ['SPOTIPY_CLIENT_SECRET'] = 'd3ebb254cbb4471d99daf76d2d37b6f0'


In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

In [14]:
# SEARCH BY NAME
from pprint import pprint
name = 'Manu Chao'
results = spotify.search(q='artist:' + name, type='artist')
uri = results['artists']['items'][0]['uri']
print('URI:', uri)

URI: spotify:artist:6wH6iStAh4KIaWfuhf0NYM


In [20]:
import pandas as pd

results = spotify.artist_albums(uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])
data = [(album['name'], album['uri'], album['total_tracks'], album['release_date'])
        for album in albums]
#for album in albums:
#    print(album['name'], album['uri'], album['total_tracks'], album['release_date'])

adf = pd.DataFrame(data, columns=['album_name', 'uri', 'nTracks', 'Released'])
adf.sort_values('Released')


,album_name,uri,nTracks,Released
6,Clandestino,spotify:album:3xoAUqjKs7Ps7wR26VAMbq,16,1998-10-06
5,Próxima Estación: Esperanza,spotify:album:4t1LLdXiWTfoywqricztFo,17,2001-06-01
4,Radio Bemba Sound System (Live),spotify:album:5zGKdwV23nlU5pnPmG1bbu,29,2002-09-17
3,Sibérie m'était contéee,spotify:album:7EZSOlWZFjsLLZkDwtFCPS,23,2004-11-01
2,La Radiolina,spotify:album:3Gyn6McG8JN6dOFeQkQX2l,21,2007-08-30
1,Baïonarena (Live),spotify:album:7i5eD0VAL2fdeUNO1IQBDj,33,2009-09-14
0,Clandestino / Bloody Border,spotify:album:09U5RBDuQaH2Lwzhocwyj6,19,2019-08-30


In [21]:
# ahora los discos
out = {}
for _, album in adf.iterrows():
    res = spotify.album_tracks(album.uri)
    out[album.album_name] = res
    print(album.album_name, len(res))

Clandestino / Bloody Border 7
Baïonarena (Live) 7
La Radiolina 7
Sibérie m'était contéee 7
Radio Bemba Sound System (Live) 7
Próxima Estación: Esperanza 7
Clandestino 7


In [31]:
def album_tracks(uri):
    tracks = spotify.album_tracks(uri)
    data = [(ri['track_number'],ri['name']) for ri in res['items']]
    return pd.DataFrame(data, columns = ['trackNo', 'name'])

In [33]:
album_tracks(adf.iloc[0]['uri'])

,trackNo,name
0,1,Clandestino
1,2,Desaparecido
2,3,Bongo Bong
3,4,Je ne t'aime plus
4,5,Mentira
5,6,Lágrimas de oro
6,7,Mama Call
7,8,Luna y sol
8,9,Por el suelo
9,10,Welcome to Tijuana


In [38]:
manu = pd.DataFrame()
for uri, aname in zip(adf.uri, adf.album_name):
    udf = album_tracks(uri)
    udf['album'] = aname
    manu = manu.append(udf)
manu

,trackNo,name,album
0,1,Clandestino,Clandestino / Bloody Border
1,2,Desaparecido,Clandestino / Bloody Border
2,3,Bongo Bong,Clandestino / Bloody Border
3,4,Je ne t'aime plus,Clandestino / Bloody Border
4,5,Mentira,Clandestino / Bloody Border
...,...,...,...
11,12,Malegría,Clandestino
12,13,La vie à 2,Clandestino
13,14,Minha galera,Clandestino
14,15,La despedida,Clandestino


In [39]:
def artist_albums(name):
    results = spotify.search(q='artist:' + name, type='artist')
    uri = results['artists']['items'][0]['uri']
    results = spotify.artist_albums(uri, album_type='album')
    albums = results['items']
    while results['next']:
        results = spotify.next(results)
        albums.extend(results['items'])
    data = [(album['name'], album['uri'], album['total_tracks'], album['release_date'])
        for album in albums]

    adf = pd.DataFrame(data, columns=['album_name', 'uri', 'nTracks', 'Released'])
    return adf
  
def artist_tracks(name):
    adf = artist_albums(name)
    manu = pd.DataFrame()
    for uri, aname in zip(adf.uri, adf.album_name):
        udf = album_tracks(uri)
        udf['album'] = aname
        manu = manu.append(udf)
    return manu

In [40]:
artist_tracks('Los Jaivas')

,trackNo,name,album
0,1,Clandestino,Jaivas En Vivo: Gira Chile '88
1,2,Desaparecido,Jaivas En Vivo: Gira Chile '88
2,3,Bongo Bong,Jaivas En Vivo: Gira Chile '88
3,4,Je ne t'aime plus,Jaivas En Vivo: Gira Chile '88
4,5,Mentira,Jaivas En Vivo: Gira Chile '88
...,...,...,...
11,12,Malegría,El Volantin
12,13,La vie à 2,El Volantin
13,14,Minha galera,El Volantin
14,15,La despedida,El Volantin
